In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os

from dotenv import load_dotenv
from dotenv import dotenv_values
from collections import namedtuple
import json
import yaml
import supervisely_lib as sly

secret_dotenv_file = 'secret_debug.env'
load_dotenv(secret_dotenv_file)
dotenv_file = 'debug.env'
load_dotenv(dotenv_file)

import sys
sys.path.append('../')
from src.bounding_box import BoundingBox
from src.utils.enumerators import BBFormat, BBType, CoordinatesType
from src.evaluators.pascal_voc_evaluator import get_pascalvoc_metrics
from src.utils.enumerators import MethodAveragePrecision

In [2]:
api: sly.Api = sly.Api.from_env()
app: sly.AppService = sly.AppService()

In [3]:
task_id = os.environ['TASK_ID']
src_project_id = os.environ['modal.state.slySrcProjectId']
dst_project_id = os.environ['modal.state.slyDstProjectId']

In [4]:
src_project = app.public_api.project.get_info_by_id(src_project_id)
if src_project is None:
    raise RuntimeError(f"Project id={src_project_id} not found")
    
dst_project = app.public_api.project.get_info_by_id(dst_project_id)
if dst_project is None:
    raise RuntimeError(f"Project id={dst_project_id} not found")

In [5]:
result = namedtuple('Result', ['TP', 'FP', 'Precision', 'Recall', 'AP'])
def dict2tuple(dictionary, round_level = 4):
    FP = 0
    TP = 0
    npos = 0
    for dict_ in dictionary['per_class']:
        dict__ = dictionary['per_class'][dict_]
        FP += dict__['total FP']
        TP += dict__['total TP']
        npos += dict__['total positives']
    AP = round(dictionary['mAP'], round_level)
    Recall = round(TP / npos, round_level)
    Precision = round(np.divide(TP, (FP + TP)), round_level)
    return result(TP, FP, Precision, Recall, AP)


def plt2bb(batch_element,
           type_coordinates = CoordinatesType.ABSOLUTE, 
           bb_type          = BBType.GROUND_TRUTH, 
           format           = BBFormat.XYX2Y2):
    # type_coordinates = CoordinatesType.X : ABSOLUTE, RELATIVE
    # bb_type          = BBType.X          : GROUND_TRUTH, DETECTED
    # format           = BBFormat.X        : XYX2Y2, XYWH, PASCAL_XML, YOLO
    ret = []
    annotations = batch_element.annotation['objects']
    for ann in annotations:
        classTitle = ann['classTitle']
        points = ann['points']['exterior']
        x1,y1 = points[0]
        x2,y2 = points[1]
        confidence = None if bb_type == BBType.GROUND_TRUTH else ann['tags'][0]['value']
        bb = BoundingBox(image_name      = batch_element.image_name,
                         class_id        = classTitle,
                         coordinates     = (x1, y1, x2, y2),
                         type_coordinates= type_coordinates,
                         img_size        = (batch_element.annotation['size']['width'], 
                                            batch_element.annotation['size']['height']),
                         confidence      = confidence,
                         bb_type         = bb_type,
                         format          = format)
        ret.append(bb)
    return ret

In [6]:
round_level = 4 # round(target_value, round_level): round(0.123456, round_level) --> 0.1235
ious = [0.5, 0.75] # list of IOU thresholds

# statistic data structure
struct = namedtuple('Set_mAP', ['name', 'mAP_05', 'mAP_075'])

# storage lists for images, datasets, projects
image_mAP   = [] # image_name   + mAP_05 + mAP_075
dataset_mAP = [] # dataset_name + mAP_05 + mAP_075
project_mAP = [] # project_name + mAP_05 + mAP_075

project_gts_bbs    = []
project_det_bbs    = []

for src, dst in zip(api.dataset.get_list(src_project.id), api.dataset.get_list(dst_project.id)):
    src_images = api.image.get_list(src.id)
    dst_images = api.image.get_list(dst.id)
    
    dataset_gts_bbs    = []
    dataset_det_bbs    = []
   
    for src_batch, dst_batch in zip(sly.batched(src_images, batch_size=10), 
                                    sly.batched(dst_images, batch_size=10)):
        src_image_ids   = [image_info.id   for image_info in src_batch]
        src_image_names = [image_info.name for image_info in src_batch]
        dst_image_ids   = [image_info.id   for image_info in dst_batch]
        dst_image_names = [image_info.name for image_info in dst_batch]
        
        src_annotations = api.annotation.download_batch(src.id, src_image_ids)
        dst_annotations = api.annotation.download_batch(dst.id, dst_image_ids)
        assert len(src_annotations)==len(dst_annotations), \
                            'Lenghst of src_annotations and dst_annotations must be the same!'
        for src_annotation, dst_annotation in zip(src_annotations, dst_annotations):
            img_gts_bbs = plt2bb(src_annotation)
            img_det_bbs = plt2bb(dst_annotation, bb_type = BBType.DETECTED)
            dataset_gts_bbs.extend(img_gts_bbs)
            dataset_det_bbs.extend(img_det_bbs)
            image_mAPs = []
            for iou in ious:
                dict_res = get_pascalvoc_metrics(
                            img_gts_bbs, img_det_bbs, iou, generate_table=True, 
                            method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
                )
                image_mAPs.append(dict_res)
                
            image_mAP.append(
                struct(
                    name    = src_annotation.image_name, 
                    mAP_05  = dict2tuple(image_mAPs[0], round_level), 
                    mAP_075 = dict2tuple(image_mAPs[1], round_level)
                )
            )
    
    project_gts_bbs.extend(dataset_gts_bbs)
    project_det_bbs.extend(dataset_det_bbs)
    
    mAPs = []
    for iou in ious:
        dict_res = get_pascalvoc_metrics(
                    dataset_gts_bbs, dataset_det_bbs, iou, generate_table=True, 
                    method=MethodAveragePrecision.ELEVEN_POINT_INTERPOLATION
        )
        mAPs.append(dict_res)
        
    dataset_mAP.append(
        struct(
            name    = src.name, 
            mAP_05  = dict2tuple(mAPs[0], round_level), 
            mAP_075 = dict2tuple(mAPs[1], round_level)
        )
    )

mAPs = []
for iou in ious:
    dict_res = get_pascalvoc_metrics(
                project_gts_bbs, project_det_bbs, iou, generate_table=True, 
                method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
    )
    mAPs.append(dict_res)
    
project_mAP.append(
    struct(
        name    = src_project.name, 
        mAP_05  = dict2tuple(mAPs[0], round_level), 
        mAP_075 = dict2tuple(mAPs[1], round_level)
    )
)

In [7]:
project_mAP

[Set_mAP(name='Lemons-aug', mAP_05=Result(TP=302.0, FP=2.0, Precision=0.9934, Recall=0.9805, AP=0.9824), mAP_075=Result(TP=294.0, FP=10.0, Precision=0.9671, Recall=0.9545, AP=0.9413))]

In [8]:
dataset_mAP

[Set_mAP(name='ds1', mAP_05=Result(TP=302.0, FP=2.0, Precision=0.9934, Recall=0.9805, AP=0.9091), mAP_075=Result(TP=294.0, FP=10.0, Precision=0.9671, Recall=0.9545, AP=0.9002))]

In [9]:
image_mAP

[Set_mAP(name='IMG_0748_05.jpeg', mAP_05=Result(TP=3.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0), mAP_075=Result(TP=3.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0)),
 Set_mAP(name='IMG_3861_08.jpeg', mAP_05=Result(TP=4.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0), mAP_075=Result(TP=4.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0)),
 Set_mAP(name='IMG_3861_06.jpeg', mAP_05=Result(TP=4.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0), mAP_075=Result(TP=4.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0)),
 Set_mAP(name='IMG_4451_07.jpeg', mAP_05=Result(TP=5.0, FP=1.0, Precision=0.8333, Recall=1.0, AP=0.975), mAP_075=Result(TP=4.0, FP=2.0, Precision=0.6667, Recall=0.8, AP=0.875)),
 Set_mAP(name='IMG_4451_09.jpeg', mAP_05=Result(TP=5.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0), mAP_075=Result(TP=5.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0)),
 Set_mAP(name='IMG_1836_05.jpeg', mAP_05=Result(TP=3.0, FP=0.0, Precision=1.0, Recall=1.0, AP=1.0), mAP_075=Result(TP=3.0, FP=0.0, Precision=1.0, Reca